# Vault Auto Unseal and Seal Wrap with SoftHSM

> THIS IS NOT A PRODUCTION SETUP BUT A DEMO: THIS IS NOT SAFE DO NOT USE
SOFTHSM TO STORE PRODUCTION KEYS: IT IS NOT SAFE DO NOT USE PKCS11PROXY IN
PRODUCTION: IT IS NOT SAFE

## Overview

This is a brief guide to exploring Vault Enterprise+HSM with SoftHSMv2, an open source software-based HSM that uses the PKCS #11 standard interface. In the guide, you’ll learn how to configure and bootstrap a minimal Vault instance that uses your newly installed and configured SoftHSM installation.

## Business Value of Capability

* Leverage an existing investment in an HSM
* Reduce risk of losing the ability to access your Vault

## Prerequisites
* **Vault Enterprise with Support for HSM (the regular Enterprise will fail**)
* Docker setup
* A Hardware Security Module (HSM) - we use SoftHSM as a proxy

### Setup Your HSM Container

In [1]:
# env
export WORKDIR=$(pwd)/config/vault/
echo $WORKDIR

/Users/pephan/OneDrive/code/vault_transit_test/hc_demos-jupyter/Vault/config/vault/


: 1

In [2]:
## tokendirectory
mkdir -p $WORKDIR/hsm

## install vault binary here
mkdir -p $WORKDIR/hsm/bin
mkdir -p $WORKDIR/hsm/tokens
mkdir -p $WORKDIR/hsm/config

: 1

In [19]:
## we create a common network
docker network create hsmnetwork && docker network ls

In [19]:
## setup softhsm container instance starting from a basic ubuntu instance
docker stop SoftHSM || true
docker rm SoftHSM
docker run \
    --rm -d \
    --name SoftHSM \
    --hostname HSM \
    --network hsmnetwork \
    -p 5666:5666 \
    -v $WORKDIR/hsm/config:/config \
    -v $WORKDIR/hsm/tokens:/var/lib/softhsm/tokens \
    bitnami/nginx:latest

## start the HRSM
# docker start SoftHSM

Error response from daemon: network with name hsmnetwork already exists
SoftHSMh
SoftHSMh
ab6615218e147b2a0f1799a87eadf1c89ff6dfb10ee98b6f6e202b33f643b2f8


: 1

## Build Scripts

There are two scripts:

* `setup_hsm.sh`: This will install the correct packages, and initialize a virtual slot we will use for Vault
* `setup_vault.sh`: This will download the trial Vault Enterprise HSM binary and configure it with systemd

### SoftHSM - Script

In [167]:
## we need to install a bunch of items
cat << EOF > $WORKDIR/hsm/config/setup_hsm.sh
#!/bin/sh
echo "--------setting up the HSM instance------------"
apt-get update -y

## install deps
echo ":... installing proxy deps before build"
apt-get install -y \
    ca-certificates \
    git-core \
    build-essential \
    cmake \
    libltdl-dev \
    libssl-dev \
    libseccomp-dev

## get the sources from github
echo ":... pulling proxy code from github"
git clone https://github.com/SUNET/pkcs11-proxy

## build pkcs11-proxy
echo ":...building the proxy..."
cd pkcs11-proxy && \
    cmake . && \
    make && \
    make install

## install softhsm2
echo ":....installing softhsm v2 and pkcs11-tool"
apt-get install -y \
    softhsm2 \
    opensc \
    gnutls-bin \
    libengine-pkcs11-openssl1.1

## install an initial token
echo ":...Initialize token in slot 0 for use with Vault."
softhsm2-util --init-token --slot 0 --label "main" \
    --pin "1234567890" \
    --so-pin "0987654321"
echo ":...launching proxy"

## launch the proxy
export PKCS11_DAEMON_SOCKET="tcp://0.0.0.0:5657"
/usr/local/bin/pkcs11-daemon /usr/lib/softhsm/libsofthsm2.so &

echo "--------setup complete------------"
EOF

printf "${GREEN}## change the file permissions${NC}\n"
chmod 755 $WORKDIR/hsm/config/setup_hsm.sh

## change the file permissions4l


: 1

### Vault - Script

In [6]:
## we need to install a bunch of items
cat << EOF > $WORKDIR/hsm/config/setup_vault.sh
#!/bin/sh

echo "--------setting up the Vault instance------------"
echo ":...setup vault data directories"

mkdir -p /home/vault
mkdir -p /home/vault/data
chmod 777 /home/vault
chmod 777 /home/vault/data

## reaching
export PKCS11_PROXY_SOCKET="tcp://HSM:5657"
export VAULT_HSM_LIB="/usr/local/lib/libpkcs11-proxy.so"
export VAULT_HSM_TYPE="pkcs11"

## install deps
echo ":... installing proxy deps before build"
apt-get update -y && \
apt-get install -y \
    ca-certificates \
    git-core \
    build-essential \
    cmake \
    libltdl-dev \
    libssl-dev \
    libseccomp-dev

## get the sources from github
echo ":... pulling proxy code from github"
git clone https://github.com/SUNET/pkcs11-proxy

## build pkcs11-proxy
echo ":...building the proxy..."
cd pkcs11-proxy && \
    cmake . && \
    make && \
    make install

## install pkcs11 tools...no need to install softhsm
echo ":....installing pkcs11-tool"
apt-get install -y \
    opensc \
    gnutls-bin \
    libengine-pkcs11-openssl1.1

echo ":...test out that you can reach the HSM slot/token"
pkcs11-tool --module=/usr/local/lib/libpkcs11-proxy.so -L

echo "--------setup complete------------"
EOF

## change the file permissions
chmod 755 $WORKDIR/hsm/config/setup_vault.sh

: 1

* PKCS related tools such as `pkcs11-tool` for debugging issues. This tool is installed as part of the `opensc` package.

## Setup the HSM

Let's run the script on the HSM to install all necessary libs

In [8]:
## run the setup script from above
docker exec -i --user root SoftHSM sh -c '/config/setup_hsm.sh'

--------setting up the HSM instance------------
Get:1 http://deb.debian.org/debian buster InRelease [122 kB]
Get:2 http://security.debian.org buster/updates InRelease [65.4 kB]
Get:3 http://deb.debian.org/debian buster/main amd64 Packages [10.7 MB]
Get:4 http://security.debian.org buster/updates/main amd64 Packages [368 kB]
Fetched 11.3 MB in 1s (8627 kB/s)
Reading package lists...
:... installing proxy deps before build
Reading package lists...
Building dependency tree...
Reading state information...
ca-certificates is already the newest version (20200601~deb10u2).
The following additional packages will be installed:
  autoconf automake autotools-dev binutils binutils-common
  binutils-x86-64-linux-gnu bzip2 cmake-data cpp cpp-8 dirmngr dpkg-dev
  fakeroot file g++ g++-8 gcc gcc-8 git-man gnupg gnupg-l10n gnupg-utils gpg
  gpg-agent gpg-wks-client gpg-wks-server gpgconf gpgsm less
  libalgorithm-diff-perl libalgorithm-diff-xs-perl libalgorithm-merge-perl
  libarchive13 libasan5 libass

: 1

The SoftHSM PKCS#11 shared library will be installed to `/usr/lib/softhsm` by default on Debian. You can do a quick verification of its presence to ensure that it's where we expect it to be:

```
docker exec -i --user root SoftHSM ls -lh /usr/lib/softhsm/
total 0
lrwxrwxrwx 1 root root 42 Mar 25  2018 libsofthsm2.so -> ../x86_64-linux-gnu/softhsm/libsofthsm2.so
```

In [ ]:
## OPTIONAL create a new slot for the unseal (only done once. if you reuse direc
# docker exec -i --user root SoftHSM sh -c 'softhsm2-util --init-token --slot 0 --

### Show current state of HSM.

In [9]:
## list the slots
docker exec -i --user root SoftHSM softhsm2-util --show-slots

Available slots:
Slot 299555966
    Slot info:
        Description:      SoftHSM slot ID 0x11dadc7e                                      
        Manufacturer ID:  SoftHSM project                 
        Hardware version: 2.4
        Firmware version: 2.4
        Token present:    yes
    Token info:
        Manufacturer ID:  SoftHSM project                 
        Model:            SoftHSM v2      
        Hardware version: 2.4
        Firmware version: 2.4
        Serial number:    602465f711dadc7e
        Initialized:      yes
        User PIN init.:   yes
        Label:            main                            
Slot 1
    Slot info:
        Description:      SoftHSM slot ID 0x1                                             
        Manufacturer ID:  SoftHSM project                 
        Hardware version: 2.4
        Firmware version: 2.4
        Token present:    yes
    Token info:
        Manufacturer ID:  SoftHSM project                 
        Model:            SoftHSM v2

: 1

### !!! Following command was truncated

In [34]:
## Select HSM Slot for Vault
VAULT_HSM_SLOT=$(docker exec -i --user root SoftHSM softhsm2-util --show-slots \
    | egrep "^Slot .." | awk '{print $NF}')
echo $VAULT_HSM_SLOT

299555966[?2004l


: 1

## Configure Vault Enterprise

## Supported Encryption/Decryption Mechanisms

A very important piece of information you need to obtain about your HSM library is the
supported mechanisms. Particularly what encryption/decryption is supported. For example -
this is the output of the `libpkcs11-proxy` library when you test logging into the slot:


```
# docker exec -it --user root SoftHSM bash
# pkcs11-tool --module=/usr/lib/softhsm/libsofthsm2.so -l -t
Using slot 0 with a present token (0x425969ad)
Logging in to "main".
Please enter User PIN:
C_SeedRandom() and C_GenerateRandom():
    seems to be OK
Digests:
    all 4 digest functions seem to work
    MD5: OK
    SHA-1: OK
Signatures: not implemented
Verify (currently only for RSA)
  testing key 0 (main)
    RSA-X-509: OK
    RSA-PKCS: OK
    SHA1-RSA-PKCS: OK
    MD5-RSA-PKCS: OK
Unwrap: not implemented
Decryption (currently only for RSA)
  testing key 0 (main)
    RSA-PKCS: OK
    RSA-X-509: OK
error: PKCS11 function C_DecryptInit failed: rv = CKR_ARGUMENTS_BAD
(0x7)
Aborting.
    RSA-PKCS-OAEP:
```

Notice

* RSA-PKCS-OAEP fails (in Vault this is mechanism 0x9) but here are Vault's supported mechanisms (notice where we match)

```
0x1085 CKM_AES_CBC_PAD (HMAC mechanism required)
0x1082 CKM_AES_CBC (HMAC mechanism required)
0x1087 CKM_AES_GCM
0x0009 CKM_RSA_PKCS_OAEP
0x0001 CKM_RSA_PKCS
```

Our only option here is `0x0001 CKM_RSA_PKCS` passed as the string "`0x1`"

As for the HMAC Mechanism we ONLY SUPPORT `CKM_SHA256_HMAC` which is passed as "`0x0251`"

Therefore we use this information in configuring Vault. We need a new config file for Vault 

Here is the config we will use to start up vault

### Vault Config File

We'll be using the most minimal non-developer-mode Vault configuration possible for this guide: 
* Run the Vault server with the `file` storage backend 
* Specifying the required HSM configuration.

Create a minimal Vault HSM configuration. Make sure to use the correct value for slot as shown in the `softhsm2-util --show-slots` command output and the correct path for your `libsofthsm2.so` file.

In [35]:
## Create the config file
cat << EOF > $WORKDIR/hsm/config/unseal.hcl
listener "tcp" {
    address = "0.0.0.0:8200"
    tls_disable = "true"
}
# Configure the storage backend for Vault
storage "file" {
    path = "/home/vault/data"
}
seal "pkcs11" {
    lib = "/usr/local/lib/libpkcs11-proxy.so"
    pin = "1234567890"
    slot = "${VAULT_HSM_SLOT}"
    key_label = "main"
    hmac_key_label = "hmac-key"
    mechanism = "0x1"
    hmac_mechanism = "0x0251"
    generate_key = "true"
}
ui = true
EOF

: 1

The storage backend is set to the local file system (`/home/vault/data`) to make the verification step easy.

The example configuration defines the following in its seal stanza:

* `lib` - Path to the PKCS #11 library on the virtual machine where Vault Enterprise is installed.
* `slot` - The slot number to use. We set this via an env var.
* `pin` - PKCS #11 PIN for login. This was set in the `setup_hsm.sh` script.
* `key_label` - Defines the label of the key you want to use.
* `hmac_key_label` - Defines the label of the key you want to use for HMACing.
* `generate_key` - If no existing key with the label specified by key_label can be found at Vault initialization time, Vault generates a key.

In [15]:
VAULT_VER=1.7.2
## pull down the vault binary needed
wget -c https://releases.hashicorp.com/vault/${VAULT_VER}+ent.hsm/vault_${VAULT_VER}+ent.hsm_linux_amd64.zip

# unzip and remove the zip file
unzip -d config/vault/hsm/bin/ vault_${VAULT_VER}+ent.hsm_linux_amd64.zip
# rm hsm/bin/vault_1.6.2+ent.hsm_linux_amd64.zip

--2021-07-01 13:21:32--  https://releases.hashicorp.com/vault/1.7.2+ent.hsm/vault_1.7.2+ent.hsm_linux_amd64.zip
Resolving releases.hashicorp.com (releases.hashicorp.com)... 151.101.1.183, 151.101.65.183, 151.101.129.183, ...
Connecting to releases.hashicorp.com (releases.hashicorp.com)|151.101.1.183|:443... connected.
HTTP request sent, awaiting response... 416 Range Not Satisfiable

    The file is already fully retrieved; nothing to do.

Archive:  vault_1.7.2+ent.hsm_linux_amd64.zip
  inflating: config/vault/hsm/bin/vault  


: 1

## Create Vault Container

In [22]:
## Vault container - in same network as the HSM
docker stop hcvault || true
docker rm hcvault
docker run \
    --rm -d \
    --name hcvault \
    --network hsmnetwork \
    --cap-add IPC_LOCK \
    -p 8200:8200 \
    -h vaultsvr \
    -e VAULT_ADDR=http://127.0.0.1:8200 \
    -v $WORKDIR/hsm/config:/config \
    -v $WORKDIR/hsm/bin:/usr/local/bin \
    bitnami/nginx:latest

Error response from daemon: No such container: hcvault
Error: No such container: hcvault
1320888b0e1ef9ef23eb71fdaa1ff6c74c728f45bc3f8e0a5ba83f862a0cea46


: 1

In [ ]:
# docker start hcvault

Verify Vault binary is an Enterprise+HSM version:

In [153]:
docker exec -it --user root hcvault vault version

Vault v1.7.2+ent.hsm (0e2ed0c723715690ce84695805e84e00466909c0) (cgo)


: 1

### hcvault

In [23]:
## run the setup script for vault from above
docker exec -i --user root hcvault sh -c '/config/setup_vault.sh'

--------setting up the Vault instance------------
:...setup vault data directories
:... installing proxy deps before build
Hit:1 http://security.debian.org buster/updates InRelease
Hit:2 http://deb.debian.org/debian buster InRelease
Reading package lists...
Reading package lists...
Building dependency tree...
Reading state information...
build-essential is already the newest version (12.6).
ca-certificates is already the newest version (20200601~deb10u2).
cmake is already the newest version (3.13.4-1).
git is already the newest version (1:2.20.1-2+deb10u3).
libseccomp-dev is already the newest version (2.3.3-4).
libltdl-dev is already the newest version (2.4.6-9).
libssl-dev is already the newest version (1.1.1d-0+deb10u6).
0 upgraded, 0 newly installed, 0 to remove and 0 not upgraded.
:... pulling proxy code from github
fatal: destination path 'pkcs11-proxy' already exists and is not an empty directory.
:...building the proxy...
-- Configuring done
-- Generating done
-- Build files ha

: 1

## Start Vault through a shell

You will need to first set the following:

In [156]:
# optional
cat <<EOF | docker exec -i --user root hcvault sh
pkill vault
rm -rf /home/vault/data
mkdir -p /home/vault/data
EOF

: 1

In [157]:
cat <<EOF | docker exec -i --user root hcvault sh
## env variables required to setup
export PKCS11_PROXY_SOCKET="tcp://HSM:5657"
export VAULT_HSM_LIB="/usr/local/lib/libpkcs11-proxy.so"
export VAULT_HSM_TYPE="pkcs11"
## then start vault with
vault server -log-level=trace -config=/config/unseal.hcl > vault.log 2>&1 &
EOF

: 1

In [25]:
## env variables required to setup
export PKCS11_PROXY_SOCKET="tcp://HSM:5657"
export VAULT_HSM_LIB="/usr/local/lib/libpkcs11-proxy.so"
export VAULT_HSM_TYPE="pkcs11"

: 1

In [ ]:
## then start vault with
vault server -log-level=trace -config=/config/unseal.hcl > vault.log 2>&1 &

### Initialize Vault

In [41]:
export VAULT_ADDR=http://127.0.0.1:8200

: 1

In [158]:
## initialize vault
vault operator init -recovery-shares=1 -recovery-threshold=1 | tee config/vault/vault.init

Recovery Key 1: 80YgzB3C748taNaNHGh/2s35tiQ5gMCTJNAhK1VAQYY=

Initial Root Token: s.Y61EPzC4Op6QhwySkBnd8c1L

Success! Vault is initialized

Recovery key initialized with 1 key shares and a key threshold of 1. Please
securely distribute the key shares printed above.


: 1

In [160]:
grep "Key 1" config/vault/vault.init | awk '{print $NF}'
grep "Token" config/vault/vault.init | awk '{print $NF}'

80YgzB3C748taNaNHGh/2s35tiQ5gMCTJNAhK1VAQYY=
s.Y61EPzC4Op6QhwySkBnd8c1L


: 1

Sample Output:
```
Recovery Key 1: jQGToHAW7R1gQXf1yJToSAZK5/2Yl8nSNS48jGOkZmM=
Initial Root Token: s.WReLx9GsdHOCVSu1w7yOc0YA
Success! Vault is initialized
Recovery key initialized with 1 key shares and a key threshold of 1. Please
securely distribute the key shares printed above.
```

NOTE: Instead of an unseal key, a recovery key is issued when using an HSM.

You can read more about Recovery Keys in the [Vault Enterprise HSM Behavioral Changes documentation](https://www.vaultproject.io/docs/enterprise/hsm/behavior.html).

In [161]:
## you can now log into vault
export VAULT_TOKEN=$(grep "Token" config/vault/vault.init | awk '{print $NF}')
# vault login $VAULT_TOKEN
vault token lookup

Key                 Value04l
---                 -----
accessor            lXNMlDH5g0Mwe2P1MTPPw5fP
creation_time       1625266460
creation_ttl        0s
display_name        root
entity_id           n/a
expire_time         <nil>
explicit_max_ttl    0s
id                  s.Y61EPzC4Op6QhwySkBnd8c1L
meta                <nil>
num_uses            0
orphan              true
path                auth/token/root
policies            [root]
ttl                 0s
type                service


: 1

## Verify - Confirm Vault will unseal using the HSM.

Stop and restart Vault.

In [163]:
# Stop Vault
# docker exec -i --user root hcvault sh -c "pkill vault"


cat <<EOF | docker exec -i --user root hcvault sh
pkill vault && sleep 2
export PKCS11_PROXY_SOCKET="tcp://HSM:5657"
export VAULT_HSM_LIB="/usr/local/lib/libpkcs11-proxy.so"
export VAULT_HSM_TYPE="pkcs11"
vault server -log-level=trace -config=/config/unseal.hcl > vault.log 2>&1 &
EOF

: 1

Confirm Vault is not Sealed.

In [53]:
## Status
vault status

Key                      Value
---                      -----
Recovery Seal Type       shamir
Initialized              true
Sealed                   false
Total Recovery Shares    1
Threshold                1
Version                  1.7.2+ent.hsm
Storage Type             file
Cluster Name             vault-cluster-8b527c6e
Cluster ID               99ff8a1a-795b-6775-4d6a-8631a1541a4a
HA Enabled               false


: 1

Sample Output:
```
Key                      Value
---                      -----
Recovery Seal Type       shamir
Initialized              true
Sealed                   false
Total Recovery Shares    1
Threshold                1
Version                  1.7.2+ent.hsm
Storage Type             file
Cluster Name             vault-cluster-8b527c6e
Cluster ID               99ff8a1a-795b-6775-4d6a-8631a1541a4a
HA Enabled               false
```

## Use an HSM to manage you Vault Master keys
## Choose your sharding options for the key
## Automatic Unseal if cluster goes down and back up (no waking sysadmins at 3am !)

## Enable Seal Wrap

To compare seal wrapped data against unwrapped data, enable key/value v1 secrets engine at two different paths: kv-unwrapped and kv-seal-wrapped.

Enable k/v v1 without seal wrap at kv-unwrapped.

In [55]:
# Enable k/v v1 without seal wrap at kv-unwrapped.
vault secrets enable -path=kv-unwrapped kv

# Enable k/v v1 with seal wrap. To do so, use the '-seal-wrap' flag when you enable the KV workflow.
vault secrets enable -path=kv-seal-wrapped -seal-wrap kv

# To enable seal wrap, pass the -seal-wrap flag when you enable a secrets engine.

In [149]:
# List the enabled secrets engines with details.
# vault secrets list -detailed -format=yaml
vault secrets list -detailed -format=table | head -n1
vault secrets list -detailed -format=table | grep -i wrap | awk -F ' ' '{print $1 "\t" $2 "\t" $3 "\t" $11 "\t" $12 "\t" $13}'
vault secrets list -detailed -format=yaml | grep -A11 wrapped

Path                Plugin       Accessor              Default TTL    Max TTL    Force No Cache    Replication    Seal Wrap    External Entropy Access    Options    Description                                                UUID
Path	Plugin	Accessor	Replication	Seal	Wrap
kv-seal-wrapped/	kv	kv_b92ab56d	n/a	6f43f084-32c3-0772-b503-2a39df5d80e9	
kv-unwrapped/	kv	kv_8358cd05	n/a	5009c5ea-1525-c79e-2fee-7238c490beab	
kv-seal-wrapped/:
  accessor: kv_b92ab56d
  config:
    default_lease_ttl: 0
    force_no_cache: false
    max_lease_ttl: 0
  description: ""
  external_entropy_access: false
  local: false
  options: null
  seal_wrap: true
  type: kv
--
kv-unwrapped/:
  accessor: kv_8358cd05
  config:
    default_lease_ttl: 0
    force_no_cache: false
    max_lease_ttl: 0
  description: ""
  external_entropy_access: false
  local: false
  options: null
  seal_wrap: false
  type: kv


: 1

Notice that the Seal Wrap parameter value is true for `kv-seal-wrapped`.

In this step, you are going to:

* Write some test data
* View the encrypted secrets

In [56]:
# Write a secret at kv-unwrapped/unwrapped for testing.

vault kv put kv-unwrapped/unwrapped password="my-long-password"

# Read the path to verify.
vault kv get kv-unwrapped/unwrapped


# Write the same secret at kv-seal-wrapped/wrapped for testing.
vault kv put kv-seal-wrapped/wrapped password="my-long-password"

# Read the path to verify.
vault kv get kv-seal-wrapped/wrapped

Success! Data written to: kv-unwrapped/unwrapped
====== Data ======?2004l
Key         Value
---         -----
password    my-long-password
Success! Data written to: kv-seal-wrapped/wrapped
====== Data ======?2004l
Key         Value
---         -----
password    my-long-password


: 1

## Test the Seal Wrap Feature

### View the encrypted secrets

The Vault server was configured to use the local file system (`/home/vault/data`) as its storage backend in this example.

```
# Configure the storage backend for Vault
storage "file" {
    path = "/home/vault/data"
}
```

Check the stored values in the `/home/vault/data/logical` directory.
* There are three sub-directories.
* One maps to `kv-unwrapped/` and another maps to `kv-seal-wrapped/` although you cannot tell by the folder names.

In [63]:
docker exec -i --user root hcvault sh -c "ls -lrtR /home/vault/data/logical"

/home/vault/data/logical:
total 12
drwx------ 2 root root 4096 Jul  1 22:28 8d01462c-f12e-0ca2-4143-0175a6a038e8
drwx------ 2 root root 4096 Jul  2 00:17 5009c5ea-1525-c79e-2fee-7238c490beab
drwx------ 2 root root 4096 Jul  2 00:17 6f43f084-32c3-0772-b503-2a39df5d80e9

/home/vault/data/logical/8d01462c-f12e-0ca2-4143-0175a6a038e8:
total 4
-rw------- 1 root root 105 Jul  1 23:05 _casesensitivity

/home/vault/data/logical/5009c5ea-1525-c79e-2fee-7238c490beab:
total 4
-rw------- 1 root root 101 Jul  2 00:17 _unwrapped

/home/vault/data/logical/6f43f084-32c3-0772-b503-2a39df5d80e9:
total 4
-rw------- 1 root root 505 Jul  2 00:17 _wrapped


: 1

View the contents of the `unwrapped` file.

In [64]:
docker exec -i --user root hcvault sh -c 'for i in $(ls -rtR /home/vault/data/logical/*/_* | grep unwrap); do cat $i; done'

{"Value":"AAAAAQJC/scotP6vQ0o85cIuf/vGUwqefLEC+SfrF1e9Kf6oo2S28SYK4P9fY/1hsAZB7r62zTdcFDB3209stg=="}


: 1

View the contents of the `wrapped` file.

In [67]:
docker exec -i --user root hcvault sh -c 'for i in $(ls -rtR /home/vault/data/logical/*/_* | grep "_wrap"); do cat $i; done'

{"Value":"ClBFfmN1YQDKByXiPOKyJn/EUSxLUOAxeDQCkIq79uh0Km421gPFvXq8z4EbnFn3mQpdwPPIKDNRqS4miOACsUAIdz8mPrmNDbDxPdPsfyKLjxIM3EfcQnJqaoqE+cS7IAEqiwIIARoEbWFpbiqAAoABMchx2+c1up4tJu1c2Dq0Af8I6b+4XbDCSMUm6xeu7MYWSQ8xaHg5ovBts6bJV29UOoALOGzSbyvwbJwEeplz94WG8gf0j2gN9U27Chjgfu0hsExX1JwHXKvmhj7/VspTIgHp6xbRwzasgMs/uZtfl+ah/jZEmrN7TtutsnFhfTEzZnYdGM8EJk+qplxcvh0FEpqXOue0Eg846VfxeLTE1kZwcZoZBL+rrVNitUqlWZbmz0bCocfPDlOQHvB4/AXz1FV1jU+SHOheTlIr6JhWDp/5VOUnQZeKGK9ZrBofyt8jPCN1o+WIM4bVGHeCJoOKWSrJV2czYHs1oko5Fxtz"}


: 1

Secrets are encrypted regardless; however, the seal-wrapped value is significantly longer despite the fact that both values are the same, `my-long-password`.

# Clean UP

# Resources

* https://learn.hashicorp.com/tutorials/vault/seal-wrap
* [SoftHSMv2 Debian Lab](https://hashicorp.atlassian.net/wiki/spaces/CSST/pages/65273946/SoftHSMv2+Debian+Lab)

* [Vault Enterprise HSM Support](https://www.vaultproject.io/docs/enterprise/hsm/index.html)
* [Vault Enterprise HSM Configuration](https://www.vaultproject.io/docs/enterprise/hsm/configuration.html)
* [SoftHSM](https://www.opendnssec.org/softhsm/)
* [SoftHSMv2 GitHub repository](https://github.com/opendnssec/SoftHSMv2)